In [156]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [157]:
# dependencies
import pymongo as pm

In [ ]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pm.MongoClient(conn)

# define db 
DB_NAME = 'scrape'
db = client[DB_NAME]

In [180]:
# this collection will store all the HTML pages
htmlCol = db[COL_NAME]
COL_NAME = 'msnbc'

# import db into local mongo instance
PATH = 'db/json/by_media/left/'
FILE = 'msnbc.json'

In [181]:
def addToDB(DB_NAME,COL_NAME,PATH,FILE):
    '''
    Imports a file into mongoDB
    
    Parameters:
    --------
    DB_NAME : Name of the database to connect to
    COL_NAME: Name of the collection to create
    PATH    : Path to folder with the file
    FILE  : Filename
    
    Returns:
    --------
    Collection COL_NAME in DB_NAME database
    '''
    !mongoimport --db {DB_NAME} --collection {COL_NAME} --file {PATH+FILE} --batchSize 1
    print(f'Collection {COL_NAME} in {DB_NAME} database created')

In [182]:
addToDB(DB_NAME,COL_NAME,PATH,FILE)

2018-06-26T21:26:04.347-0400	connected to: localhost
2018-06-26T21:26:04.378-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b243fb2897e82028b8ffe50') }
2018-06-26T21:26:04.380-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b243fb7897e82028b8ffe51') }
2018-06-26T21:26:04.384-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b243fbc897e82028b8ffe52') }
2018-06-26T21:26:04.387-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b243fc1897e82028b8ffe53') }
2018-06-26T21:26:04.394-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b243fc6897e82028b8ffe54') }
2018-06-26T21:26:04.396-0400	error inserting documents: E11000 duplicate key error coll

2018-06-26T21:26:04.560-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b2440f5897e82028b8ffe88') }
2018-06-26T21:26:04.562-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b2440df897e82028b8ffe84') }
2018-06-26T21:26:04.565-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b244100897e82028b8ffe8a') }
2018-06-26T21:26:04.568-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b2440e4897e82028b8ffe85') }
2018-06-26T21:26:04.571-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b24410a897e82028b8ffe8c') }
2018-06-26T21:26:04.574-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectI

2018-06-26T21:26:04.712-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b244262897e82028b8ffec4') }
2018-06-26T21:26:04.714-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b244269897e82028b8ffec5') }
2018-06-26T21:26:04.714-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b24427e897e82028b8ffec9') }
2018-06-26T21:26:04.717-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b244279897e82028b8ffec8') }
2018-06-26T21:26:04.722-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b24426e897e82028b8ffec6') }
2018-06-26T21:26:04.726-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectI

2018-06-26T21:26:04.871-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b2443ca897e82028b8ffefd') }
2018-06-26T21:26:04.873-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b2443e7897e82028b8fff01') }
2018-06-26T21:26:04.879-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b2443d5897e82028b8ffeff') }
2018-06-26T21:26:04.882-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b2443e2897e82028b8fff00') }
2018-06-26T21:26:04.885-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b2443ec897e82028b8fff02') }
2018-06-26T21:26:04.886-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectI

2018-06-26T21:26:05.023-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b244528897e82028b8fff37') }
2018-06-26T21:26:05.027-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b24455a897e82028b8fff40') }
2018-06-26T21:26:05.028-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b244560897e82028b8fff41') }
2018-06-26T21:26:05.028-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b244565897e82028b8fff42') }
2018-06-26T21:26:05.031-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b24454f897e82028b8fff3e') }
2018-06-26T21:26:05.032-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectI

...

2018-06-26T21:26:07.632-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b246096897e82028b900391') }
2018-06-26T21:26:07.634-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b24608a897e82028b90038f') }
2018-06-26T21:26:07.635-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b2460a1897e82028b900393') }
2018-06-26T21:26:07.640-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b246090897e82028b900390') }
2018-06-26T21:26:07.643-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b24609c897e82028b900392') }
2018-06-26T21:26:07.644-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectI

2018-06-26T21:26:07.787-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b246232897e82028b9003d3') }
2018-06-26T21:26:07.793-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b24623e897e82028b9003d5') }
2018-06-26T21:26:07.798-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b246249897e82028b9003d7') }
2018-06-26T21:26:07.804-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b246254897e82028b9003d9') }
2018-06-26T21:26:07.805-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b24624e897e82028b9003d8') }
2018-06-26T21:26:07.808-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectI

2018-06-26T21:26:07.943-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b24637d897e82028b900405') }
2018-06-26T21:26:07.944-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b2463b0897e82028b90040d') }
2018-06-26T21:26:07.948-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b2463b6897e82028b90040e') }
2018-06-26T21:26:07.949-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b2463ce897e82028b900412') }
2018-06-26T21:26:07.953-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b2463bc897e82028b90040f') }
2018-06-26T21:26:07.954-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectI

2018-06-26T21:26:08.150-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b2464ed897e82028b900440') }
2018-06-26T21:26:08.152-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b246506897e82028b900443') }
2018-06-26T21:26:08.163-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b2464fa897e82028b900441') }
2018-06-26T21:26:08.165-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b24650c897e82028b900444') }
2018-06-26T21:26:08.168-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectId('5b246500897e82028b900442') }
2018-06-26T21:26:08.169-0400	error inserting documents: E11000 duplicate key error collection: scrape.msnbc index: _id_ dup key: { : ObjectI

Collection msnbc in scrape database created


In [160]:
def feature_extractor(html):
    '''
    Parse html using newspaper
    
    Parameters:
    --------
    html   : 'string'
    
    Returns:
    --------
    result : dictionary
    
    '''
    import newspaper
    from bs4 import BeautifulSoup as bs
    from datetime import datetime
    
    article = newspaper.Article('')
    article.set_html(html)
    article.build()
    
    # parse date manually if it wasn't found by newspaper
    if not article.publish_date:
        sp = bs(html, 'lxml')
        # 'dek___3AQpw' class appears on 30% of msnbc websites
        try:
            publish_date = datetime.strptime(\
                               (sp.find('p', class_='dek___3AQpw').span.text),\
                               '%b.%d.%Y'\
                            )
        except:
            publish_date = ''
    else:
        publish_date = article.publish_date


    return {
            'date'    :publish_date,
            'title'   :article.title,
            'text'    :article.text,
            'authors' :article.authors,
            'keywords':article.keywords
    }

In [175]:
def docs_parser(htmlCol):
    '''
    Parse mongo docs, extract features and update the doc with the features
    
    Parameters:
    --------
    htmlCol : mongodb collection, has to have documents with 'html' key
    
    Returns:
    --------
    updates all documents in the collection
    '''

    for doc in htmlCol.find():
        print(f"{doc['_id']}:")
            
        if 'html' in doc:
            # extract metadata from html
            meta = feature_extractor(doc['html'])
            
            try:
                if meta == doc['meta']:
                    print(f"has same meta")
            except:
                # if there is newer meta data or meta key is not existing
                htmlCol.update_one(
                    {'_id':ObjectId(doc['_id'])},
                    {'$set' : {
                              'meta' : meta
                              }
                    }
                )
                print(f"updated meta")
        else:
            print(f"does not have html")
        print('----------')

In [162]:
def show_doc(id):
    '''
    Finds a document by 'id' and prints contents to the console
    
    Parameters:
    --------
    id : mongodb document id
    
    Returns:
    --------
    Prints first 100 symbols of each document's key to console
    '''
    from bson.objectid import ObjectId
    doc = htmlCol.find_one({'_id':ObjectId('5b243fbc897e82028b8ffe52')})
    for k in doc:
        print(f"{k} : {str(doc[k])[:100]}")

Some comands to keep dbs clean

In [98]:
# deletes all 'meta' fields from all docs
# htmlCol.update({}, {$unset: {meta:1}}, false, true); # mongo shell comand
htmlCol.update({}, {'$unset': {'meta':1}}, multi=True) # pymongo way


/Users/yegor3/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  This is separate from the ipykernel package so we can avoid doing imports until


{'n': 1586, 'nModified': 1586, 'ok': 1.0, 'updatedExisting': True}

In [105]:
# leaves only unique documents by 'url' field

htmlCol.create_index(
    "url",
    unique=True
)

'url_1'

In [ ]:
# pymongo 'find' returns cursor that allows iterating through results
# calling first object [0] allows accessing the dictionary with results
# the ['html'] is the key in the dictionary
html = htmlCol.find({'url':'http://www.msnbc.com/velshi-ruhle/watch/jeff-sessions-is-justifying-harsh-immigration-policy-with-the-bible-1256689731629'},\
            projection={'html':True, '_id':False})[0]['html']

In [127]:
# find documents NOT containing a 'tag': regex expression
import re
tag = re.compile('dek___3AQpw.')
docs = htmlCol.find({"html" : {'$not': tag}})
for d in docs[:20]: print(d['url'])


http://www.msnbc.com/
http://www.msnbc.com/{{path.prefix}}/transcripts
http://www.msnbc.com/home2?page=1
http://www.msnbc.com/rachel-maddow-show/trump-jr-offers-the-wrong-response-concerns-the-gop-cult
http://www.msnbc.com/explore
http://www.msnbc.com/rachel-maddow-show/president-tries-brush-lying-about-infamous-trump-tower-meeting
http://www.msnbc.com/#main-menu
http://www.msnbc.com/{{path.prefix}}/transcripts#main-menu
https://www.facebook.com/sharer/sharer.php?u=https://www.msnbc.com/the-last-word/watch/lawrence-trump-tries-to-steal-the-grief-of-fallen-soldiers-parents-1256235075972
http://www.msnbc.com/rachel-maddow-show/the-good-question-team-trump-considers-ridiculous-and-ludicrous
http://www.msnbc.com/guns
https://twitter.com/intent/tweet?text=Trump%20calls%20IG%20report%20'horror%20show'%2C%20blames%20Dems%20for%20separations&via=msnbc&url=https://www.msnbc.com/morning-joe/watch/trump-calls-ig-report-horror-show-blames-dems-for-separations-1256477251672&original_referer=https:/

In [155]:
??test

In [ ]:
test = newspaper.Article(url='http://www.msnbc.com/rachel-maddow-show/defamation-lawsuit-grows-more-serious-trump')
test.build()

In [146]:
print(test.publish_date)

None


In [150]:
test.date

AttributeError: 'Article' object has no attribute 'date'

In [147]:
test.meta_data

defaultdict(dict,
            {'og': {'url': 'http://www.msnbc.com/rachel-maddow-show/defamation-lawsuit-grows-more-serious-trump',
              'type': 'article',
              'site_name': 'MSNBC',
              'title': 'Defamation lawsuit grows more serious for Trump',
              'description': "As if Donald Trump's lawyers weren't already busy, Summer Zervos' civil suit creates a real threat -- which isn't going away.",
              'image': 'http://www.msnbc.com/sites/msnbc/files/styles/ratio--1_91-1--1200x630/public/06525535.jpg?itok=lrFK_ych'},
             'twitter': {'card': 'summary_large_image',
              'site': '@msnbc',
              'creator': {'identifier': '@msnbc', 'id': 2836421},
              'url': 'http://www.msnbc.com/rachel-maddow-show/defamation-lawsuit-grows-more-serious-trump',
              'account_id': 2836421,
              'domain': 'msnbc.com',
              'title': 'Defamation lawsuit grows more serious for Trump',
              'description

**Production code**

In [176]:
docs_parser(htmlCol)

5b243fb2897e82028b8ffe50:
has same meta
----------
5b243fb7897e82028b8ffe51:
has same meta
----------
5b243fbc897e82028b8ffe52:
has same meta
----------
5b243fc1897e82028b8ffe53:
has same meta
----------
5b243fc6897e82028b8ffe54:
has same meta
----------
5b243fd0897e82028b8ffe55:
has same meta
----------
5b243fd5897e82028b8ffe56:
has same meta
----------
5b243fda897e82028b8ffe57:
has same meta
----------
5b243ff1897e82028b8ffe5b:
has same meta
----------
5b243fec897e82028b8ffe5a:
has same meta
----------
5b243fe0897e82028b8ffe58:
has same meta
----------
5b243fe7897e82028b8ffe59:
has same meta
----------
5b244001897e82028b8ffe5e:
has same meta
----------
5b243ff6897e82028b8ffe5c:
has same meta
----------
5b243ffc897e82028b8ffe5d:
has same meta
----------
5b244007897e82028b8ffe5f:
has same meta
----------
5b244011897e82028b8ffe61:
has same meta
----------
5b24401b897e82028b8ffe63:
has same meta
----------
5b24400c897e82028b8ffe60:
has same meta
----------
5b244025897e82028b8ffe65:
has s

KeyboardInterrupt: 

In [122]:
show_doc('5b243fb2897e82028b8ffe50')

_id : 5b243fbc897e82028b8ffe52
url : https://www.msnbc.com/all-in/watch/white-house-can-t-fill-open-positions-turns-to-job-fair-125617056
html : <!DOCTYPE html><html lang="en" data-reactroot=""><head><title data-rh="true">White House can&#x27;t 
